In [1]:
import tweepy
import pandas as pd
import datetime as datetime

auth = tweepy.OAuth2BearerHandler("AAAAAAAAAAAAAAAAAAAAAH6nYwEAAAAAK%2BezpbKj7NUWoAMW6SbFOfFruD4%3Dj134xtdV6I1kcGl4a1wnAaa7N2W0hQCA9avu7sDyvEEVWjIwmy")
api = tweepy.API(auth)

In [ ]:
#Searching the archive for tweets mentioning the mavs

mavsSearch = api.search_full_archive('dev', 'mavs', maxResults = 100)

In [ ]:
#Viewing the first tweet

import json
status = mavsSearch[0]
json_str = json.dumps(status._json)
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

In [2]:
#Creates the mineTweet function which extracts the information from a query and returns a dictionary with tweet data
#BUG: hashtags label also collects other random data from entities

def mineTweets(query, fromDate=datetime.datetime.now()):
        '''
        Pulls tweets from the twitter api given a query and latest date.
        Reurns a dataframe.
        '''
        data = []
        fromDate = datetime.datetime.strptime(str(fromDate), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')

        searchResult = api.search_full_archive('dev', query, fromDate=fromDate, maxResults = 100)
        for item in searchResult:
                mined = {
                        'tweet_id':        item.id,
                        'name':            item.user.name,
                        'screen_name':     item.user.screen_name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.text,
                        'hashtags':        item.entities['hashtags'],
                        'location':        item.place,
                        'is_verified':     item.user.verified,
                        'created_at':      item.created_at
                }
                try:
                        mined['retweet_text'] = item.retweeted_status.full_text
                except:
                        mined['retweet_text'] = 'None'
                try:
                        mined['quote_text'] = item.quoted_status.full_text
                        mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                        mined['quote_text'] = 'None'
                        mined['quote_screen_name'] = 'None'
                
                data.append(mined)
        return pd.DataFrame(data)

In [ ]:
#Mining tweets on the mavs, commented out because twitter access was removed due to too many requests for the day.

# mavs_df = mineTweets('mavs', '202202100019')
# mavs_df = pd.DataFrame()

# time = datetime.datetime.now()
# time = datetime.datetime.strptime(str(time), "%Y-%m-%d %H:%M:%S.%f").strftime('%Y%m%d%H%M')
# time = int(time)

# while time > 202202090019:
#     mavs_df = mavs_df.append(mineTweets('mavs', time))
    
#     dtTime = datetime.datetime.strptime(str(time), "%Y%m%d%H%M%S")
#     if mavs_df.tweet_id.duplicated().sum() > 0:
#         dtTime = dtTime - datetime.timedelta(minutes=5)
#     else:
#         dtTime = dtTime - datetime.timedelta(minutes=10)
#     time = int(dtTime.strftime('%Y%m%d%H%M'))

# mavs_df

NameError: name 'mineTweets' is not defined

In [19]:
#Trying SNScrape Since the twitter API rate limited the scraper

import snscrape.modules.twitter as sntwitter

def mineSntwitter(query, maxCount):
    '''
    Pulls tweets given a query and max amount of tweets to pull. Query can accept anything that twitter search function accepts. 
    Returns a dataframe.
    '''
    tweet_list = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i>maxCount:
            break
        tweet_list.append([
                    tweet.date,
                    tweet.id,
                    tweet.user.username,
                    tweet.content,
                    tweet.user.verified,
                    tweet.retweetCount,
                    tweet.likeCount,
        ])
        tweet_df = pd.DataFrame(tweet_list, columns=['Datetime','tweet_id', 'username', 'text', 'verified', 'retweet_count', 'like_count'])

    return tweet_df

In [5]:
#Using SNScrape to scrape on tweets mentioning the mavs for the 2021-22 Season
#TODO: Scrape for a list of queries relating to the mavs/whatever team and then drop the duplicates

mavs_df = mineSntwitter('mavs since:2021-10-20 until:2022-02-10', 100)

mavs_df.to_csv('mavs1.csv', index=False)

In [7]:
mavs_df

,Datetime,tweet_id,username,text,verified,retweet_count,like_count
0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,@603live I have been told the Mavs and Bulls a...,False,0,1
1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,@SpotOn96057010 @RGutman_ @Mavs_FFL Your first...,False,0,1
2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade for Hield, Nurkic, you know players that...",False,0,1
3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"@cazefontoura Ja hj ,ninguém sabe quem é. \nSo...",False,0,4
4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview: Jason Kidd, Trey Bur...",False,0,0
...,...,...,...,...,...,...,...
96,2022-02-09 23:35:44+00:00,1491556452196130819,shektheshrek,Mavs Fans!! Last home game before All Star bre...,False,0,1
97,2022-02-09 23:35:23+00:00,1491556363788500992,Schlitzstop,@All_Things_Mavs Is our powder still dry? For ...,False,0,0
98,2022-02-09 23:34:25+00:00,1491556118929289217,themavsoutsider,"@Mavs_FFL Give Detroit 2 firsts, those picks w...",False,0,0
99,2022-02-09 23:34:16+00:00,1491556082749218816,RGutman_,@Mavs_FFL THJ + Smith + 1st round pick\ndone d...,False,0,0


In [ ]:
date_df = pd.read_csv('datelist.csv')
dateSlice = date_df['date'][56:]

In [59]:
#Looping through dates to scrape tweets through a larger time period
date_df = pd.read_csv('datelist.csv')

#loops starting from the 56th column
dateSlice = date_df['date'][56:]
for day in dateSlice:
    mavs_df2 = pd.concat([mavs_df, mineSntwitter('mavs since:2021-10-20 until:' + day, 6000)])

mavs_df2.to_csv('mavs_df2.csv', index=False)


In [62]:
mavs_df2

,Datetime,tweet_id,username,text,verified,retweet_count,like_count
0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,@603live I have been told the Mavs and Bulls a...,False,0,1
1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,@SpotOn96057010 @RGutman_ @Mavs_FFL Your first...,False,0,1
2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade for Hield, Nurkic, you know players that...",False,0,1
3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"@cazefontoura Ja hj ,ninguém sabe quem é. \nSo...",False,0,4
4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview: Jason Kidd, Trey Bur...",False,0,0
...,...,...,...,...,...,...,...
1836,2021-10-20 00:02:19+00:00,1450613312778031108,ChrisGallegosNV,@MensaNinja @Mavs_FFL I’m not saying throw the...,False,0,0
1837,2021-10-20 00:01:13+00:00,1450613037568872454,MensaNinja,@dallassportzone @isaactweeting @All_Things_Ma...,False,0,0
1838,2021-10-20 00:01:02+00:00,1450612991649632262,ZachPrell,"@NickGamerForceJ I sure hope so, just want to ...",False,0,1
1839,2021-10-20 00:00:54+00:00,1450612957193261056,Aaron_Del24,@KillSwitchTD92 @rawljul Jazz\nSuns\nLakers\nM...,False,0,0


In [71]:
# Merging the two dataframes and dropping any duplicate tweets

mavs_df = pd.read_csv('mavs_df.csv')
mavs_df2 = pd.read_csv('mavs_df2.csv')

mavs_df_all = pd.concat([mavs_df, mavs_df2])
mavs_df_all = mavs_df_all.drop_duplicates(subset=['tweet_id'])
mavs_df_all = mavs_df_all.reset_index(drop=True)
mavs_df_all
mavs_df_all.to_csv('mavs_tweets_all.csv', index=False)
